In [1]:
# %load symmetric_encryption.py
#!/usr/bin/env/python3

import urllib.request
from Crypto.Cipher import AES
import binascii
import base64
import random
import os
import zlib

################################################################################
# CS 284 Padding Utility Functions
################################################################################

# s is a bytearray to pad, k is blocklength
# you won't need to change the block length
def cmsc284pad(s,k=16):
  if k > 255:
    print("pkcs7pad: padding block length must be less than 256")
    return bytearray()
  n = k - (len(s) % k)
  if n == 0:
    n = k
  for i in range(1,n+1):
    s.extend([i])
  return s

# s is bytes to pad, k is blocklength
# you won't need to change the block length
def cmsc284padbytes(s,k=16):
  if k > 255:
    raise Exception("pkcs7pad: padding block length must be less than 256")
  n = k - (len(s) % k)
  if n == 0:
    n = k
  for i in range(1,n+1):
    s += chr(i).encode("utf-8")
  return s

# s is bytes to unpad, k is blocklength
# you won't need to change the block length
def cmsc284unpad(s,k=16):
  if not cmsc284checkpadding(s,k):
    print("cmsc284unpad: invalid padding")
    return b''
  n = s[len(s)-1]
  return s[:len(s)-n]

# checks padding on s and returns a boolean
# you won't need to change the block length
def cmsc284checkpadding(s,k=16):
  if(len(s) == 0):
    #print("Invalid padding: String zero length"%k) 
    return False
  if(len(s)%k != 0): 
    #print("Invalid padding: String is not multiple of %d bytes"%k) 
    return False
  n = s[len(s)-1]
  if n > k or n == 0:
    return False
  else: 
    for i in range(n):
      if s[len(s)-1-i] != (n-i):
        return False
  return True

################################################################################
# Function for querying the server
################################################################################

PPS2SERVER = "http://cryptoclass.cs.uchicago.edu/"
def make_query(task, cnetid, query):
  DEBUG = False
  if DEBUG:
    print("making a query")
    print("Task:", task)
    print("CNET ID:", cnetid)
    print("Query:", query)
  if (type(query) is bytearray) or (type(query) is bytes):
    url = PPS2SERVER + urllib.parse.quote_plus(task) + "/" + urllib.parse.quote_plus(cnetid) + "/" + urllib.parse.quote_plus(base64.urlsafe_b64encode(query)) + "/"
  else:
    url = PPS2SERVER + urllib.parse.quote_plus(task) + "/" + urllib.parse.quote_plus(cnetid) + "/" + urllib.parse.quote_plus(base64.urlsafe_b64encode(query.encode('utf-8'))) + "/"
  if DEBUG:
    print("Querying:", url)

  with urllib.request.urlopen(url) as response:
    raw_answer = response.read()
    answer = base64.urlsafe_b64decode(raw_answer)
    if DEBUG:
      print("Answer:", answer)
    return answer
  return None

In [6]:
from collections import Counter
import math

In [3]:
################################################################################
# Problem 1 SOLUTION
################################################################################

def problem1(cnetid):
  flag_len = len(make_query('one', cnetid, b''))
  bias_idx = 30

  flag_ints = []
  for i in range(flag_len):
    zeros_query = bytearray(bias_idx - i)
    flag_candidates = []
    for _ in range(150):
      ctext = make_query('one', cnetid, zeros_query)
      flag_candidates.append(ctext[bias_idx])
    counts = Counter(flag_candidates)
    byte_int = counts.most_common(1)[0][0]
    flag_ints.append(byte_int)
    print(i, bytes([byte_int]))
  return bytes(flag_ints)

In [4]:
problem1('alex8')

0 b'H'
1 b'a'


KeyboardInterrupt: 

In [8]:
################################################################################
# Problem 2 SOLUTION
################################################################################

def problem2(cnetid):
  ctext_a = make_query('twoa', cnetid, '')
  ctext_prof = ctext_a[32:]
  query_b = b'davidcausername=' + bytes(cnetid, 'utf-8') + b'&uid=8'
  ctext_b = make_query('twob', cnetid, query_b)
  query_c = ctext_b[16:48] + ctext_prof
  return make_query('twoc', 'ruolinzheng', query_c)

In [9]:
problem2('ruolinzheng')

b'Admin access granted.'

In [5]:
################################################################################
# Problem 3 SOLUTION
################################################################################

def get_flag_len(cnetid):
  """get unpadded flag len"""
  padded_flag_len = len(make_query('three', cnetid, ''))
  flag_len = -1
  prev = -1
  for i in range(1, 17):
    curr = len(make_query('three', cnetid, bytes(i)))
    if prev == padded_flag_len and curr == padded_flag_len + 16:
      flag_len = padded_flag_len - i
      break
    prev = curr
  return flag_len

def get_byte_map(prev_bytes):
  # should feed this function prev_bytes[:16]
  if len(prev_bytes) > 16:
    raise
  mp = {}
  for i in range(256):
    query_bytes = bytes([i]) + prev_bytes
    query = cmsc284padbytes(query_bytes)
    ctext = make_query('three', 'ruolinzheng', query)
    # always only need the first block
    mp[ctext[:16]] = query_bytes
  return mp

def get_block_loc(flag_len):
  """get start and end pos of the target block"""
  block_idx = math.ceil((flag_len - 1) / 16)
  start = 16 * block_idx
  return start, start + 16

def get_first_query_len(flag_len):
  """get the length of the first query, [0, 15]"""
  return (16 - (flag_len - 1) % 16) % 16

def problem3(cnetid):
  flag_len = get_flag_len(cnetid)
  start, end = get_block_loc(flag_len)
  first_query_len = get_first_query_len(flag_len)
  prev_bytes = b''
  for i in range(flag_len):
    byte_mp = get_byte_map(prev_bytes[:16])
    query = bytes(first_query_len + len(prev_bytes))
    ctext = make_query('three', 'ruolinzheng', query)
    new_bytes = byte_mp[ctext[start : end]]
    prev_bytes = bytes([new_bytes[0]]) + prev_bytes
    print(i, prev_bytes)
  return prev_bytes

In [8]:
problem3('ruolinzheng')

0 b' '


KeyboardInterrupt: 

In [ ]:
################################################################################
# Problem 4 SOLUTION
################################################################################

def problem4(cnetid):
  return b''

In [ ]:
################################################################################
# Problem 5 SOLUTION
################################################################################

def problem5(cnetid):
  return b''

In [ ]:
################################################################################
# Problem 6 SOLUTION
################################################################################

def problem6(cnetid):
  return b''

In [7]:
# example running AES; delete the code below here
key = b'ABCDEFGHABCDEFGH'
block1 = b'abcdefghabcdefgh'
block2 = bytearray(b'abcdefghabcdefgh')

# we declare the mode to be ECB but can just it or single-block calls to
# AES
cipher = AES.new(key, AES.MODE_ECB)
print(cipher.encrypt(block1))

# the following call with fail without the converting block2 to bytes the
# call to AES. The AES implementation requires an immutable object and
# bytearray is mutable. Same goes for key.
print(cipher.encrypt(bytes(block2)))

# test query, will hang if off campus
print(make_query('one','davidcash', ''))

# bytearrays are mutable, which is handy
print(block2)
block2.extend([0])
print(block2)
block2.extend(block1)
# block2 = bytearray('abcdefghabcdefgh', 'utf-8')
print(block2)

b'\n\x9a\xf0C\x18#\x8bi;;\xc0\xb1~\xee"s'
b'\n\x9a\xf0C\x18#\x8bi;;\xc0\xb1~\xee"s'
b'\x9e\xde\x8e\x8d;\xab]\xcc\xac\xcb<\x81\xb1\xe18\xc9Y'
bytearray(b'abcdefghabcdefgh')
bytearray(b'abcdefghabcdefgh\x00')
bytearray(b'abcdefghabcdefgh\x00abcdefghabcdefgh')
